<a id="top"></a>
# Searching MAST using astroquery.mast's Mast class.
***
## Learning Goals

By the end of this tutorial, you will:

- Recognize the types of services MAST Portal API provides through `astroquery.mast` and their required parameters.
- Understand the benefits of `astroquery.mast`'s `Mast` class.
- Make queries for different types of services using `astroquery.mast.Mast.mast_query`.


## Introduction
`astroquery.mast.Mast` is a class that allows programmatic access to the MAST Portal. It is similar to `astroquery.mast.Observations` in that the Observations class also allows direct programmatic access to the MAST Portal. The Mast class is still needed in addition to Observations because Observations provides simplified methods to query MAST's CAOM database, while Mast can be used to query other MAST databases including CAOM. Compared to Observations, using Mast, users can create more refined queries using science instrument keywords. 

Currently users who want to query for observations using the Mast class must compile their own JSON/dictionaries to feed into the MAST Portal API. To simplify queries, users should use the `mast_query` method under `astroquery.mast.Mast`. 


The workflow for this notebook consists of:
* [Filtered Queries](#Filtered-Queries)
    * [JWST Query](#JWST-Query)
    * [TESS Query](#TESS-Query)
* [Cone Search](#Cone-Search)
* [Exercises](#Exercises)
* [Additional Resources](#Additional-Resources)

## Imports

- *astropy.units* to handle unit conversion.
- *astropy.time Time* to handle time format conversion.

In [ ]:

import astropy.units as u

from astropy.time import Time
from astroquery.mast import Mast
from astroquery.mast import Observations

## Date Format Helper Method

In [ ]:
def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }

***

## Filtered Queries

MAST's Filtered services use the parameters 'columns' and 'filters'. The 'columns' parameter is a required string that specifies the columns to be returned as a comma-separated list. The 'filters' parameter is a required list of filters to be applied. Using the `mast_query` method, users can list filters using keyword arguments and a list of values. The examples below will show the necessary format of these arguments for different services.

### JWST Query 

This is a query to view all targets of opportunity observed this year so far by JWST's Nirspec instrument. The table only shows the specified columns.

In [ ]:
jwst_table = Mast.mast_query('Mast.Jwst.Filtered.Nirspec',
                        columns = 'title, instrume, targname, date_obs_mjd',
                        targoopp = ['T'],
                        date_obs_mjd = [set_mjd_range('2023-01-01', '2023-08-01')]
                       )

To verify that the query above returns the correct range of observations, convert the time format form mjd to iso for readability.

In [ ]:
jwst_table['date_obs_mjd'] = Time(jwst_table['date_obs_mjd'], format='mjd').iso

In [ ]:
# Output the table.
jwst_table

### TESS Query

TESS queries have 2 types of filtered services. There are [Mast.Catalogs.Filtered...](https://mast.stsci.edu/api/v0/_services.html#MastCatalogsFilteredTic) and [Mast.Catalogs.Filtered...Rows](https://mast.stsci.edu/api/v0/_services.html#MastCatalogsFilteredTicRows) services. To specify columns using the columns parameter in the `mast_query` method, 'rows' services must be used and the other must be used to count the number of returned methods. 

In [ ]:
# Example of a 'rows' service
Mast.mast_query('Mast.Catalogs.Filtered.Tic.Position.Rows',
           columns = 'id, objtype, lumclass, ra, dec, Teff, logg',
           Teff = [{'min': 4250, 'max': 4500}], 
           logg = [{'min': 4.5, 'max': 5.0}],
           ra = 210.8023,
           dec = 54.349,
           radius = .2
          )

In [ ]:
# Example of a service without 'rows'
Mast.mast_query('Mast.Catalogs.Filtered.Tic.Position',
           columns = 'COUNT_BIG(*)',
           Teff = [{'min': 4250, 'max': 4500}],
           logg = [{'min': 4.5, 'max': 5.0}],
           ra = 210.8023,
           dec = 54.349,
           radius = .2
          )

## Cone Search

The MAST Portal API's cone searches are not filtered using non-positional criteria. Most cone searches only accept the parameters 'ra', 'dec', and radius and return a table with all columns present. Users can limit the columns by assigning the table returned from the `mast_query` call to a variable and masking it with a list of column names.

In [ ]:
gaia_table = Mast.mast_query('Mast.Catalogs.GaiaDR1.Cone',
                        ra = 254.287,
                        dec = -4.09933,
                        radius = 0.02
                       )

In [ ]:
# Output the table
gaia_table

Even though the radius is specified in degrees, the resulting table lists it in arc minutes under the 'distance' column. To verify that the query above returns the correct observation, convert the distance values to degrees using the 'astropy.units' module and make sure they're less than the given radius.

In [ ]:
(max(gaia_table['distance']) * u.arcmin).to(u.deg)

## Additional Resources

- [MAST Services](https://mast.stsci.edu/api/v0/_services.html) 
- [astroquery.mast Documentation](https://astroquery.readthedocs.io/en/latest/mast/mast_mastquery.html)

## Citations

If you use `astropy` or `astroquery` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`astroquery`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)



## About this Notebook

**Author(s):** Makayla St. Cyr <br>
**Keyword(s):** Tutorial, MAST, TESS, JWST, astroquery, cone <br>
**Last Updated:** Aug 2023 <br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 